# Wildcard Handling with Astroquery.mast

----

## Learning Goals

By the end of this tutorial, you will:

* Use the wildcards available for `astroquery.mast.Observations` criteria queries
* Broaden and refine `astroquery.mast.Observations` criteria queries
* Fully utilize the `instrument_name` criteria, especially for JWST queries
* Query for moving targets using target ephemeris and time criteria such as `t_min` and `t_max`

## Introduction

This Notebook demonstrates the use of wildcards in `astroquery.mast.Observations` criteria queries. The use of wildcards is encouraged for certain criteria types (namely, `string` object types) to ensure your query returns all results. 

We will demonstrate 3 use-cases for wildcards when doing criteria queries and emphasize certain criteria where wildcard usage is highly encouraged, particularly for JWST queries. We will also use the last example to demonstrate the use of value ranges when working with `float` object criteria types.

The workflow for this notebook consists of:

* [Wildcard overview with `astroquery.mast.Observations`](#overview)
    1. [Wildcard Search with `instrument_name`](#case1)
    2. [Wildcard Search with `instrument_name` and `proposal_id`](#case2)
    3. [Wildcard Search a Time-sensitive Object with `target_name` and `t_min`](#case3)
* Resources

## Imports

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt

from astropy.coordinates import SkyCoord
from astropy.table import Table, unique, vstack
from astropy.time import Time
from astroquery.mast import Observations

----

<a id="overview"></a>
## Wildcards with `astroquery.mast.Observations`

The use of wildcards when making `astroquery.mast.Observations` queries can help ensure you retrieve all observations without leaving anything out. The available wildcards are `%` and `*`: `%` replaces a single character, while `*` replaces more than one character preceding, following, or in between the existing characters, depending on its placement. See the [Observation Criteria Queries](https://astroquery.readthedocs.io/en/latest/mast/mast.html#observation-criteria-queries) section in the `astroquery.mast` documentation for more information on the wildcards.

Wildcards are only available for certain criteria. `string` type objects accept wildcards, but `float`, `integer`, or any other objects do not accept wildcards.

Users may call the `get_metadata` method to see the list of query criteria and their data types. The criteria listed as `string` objects under the **Data Type** column are criteria that can be called with wildcards:

In [ ]:
Observations.get_metadata("observations").show_in_notebook()

<a id="case1"></a>
### Case 1: Wildcard Search with `instrument_name`

For our first example we will search for all NIRISS observations taken by a certain proposal/program PI. Our two query criteria are `proposal_pi` and `instrument_name`, which are both `string` object criteria. As such, both can be wildcarded for ease of use. 

In fact, it is sometimes necessary to use wildcards when searching on `instrument_name`. Both HST and JWST use instrument configurations in this field to allow for more precise advanced searches (e.g. NIRISS/IMAGE and STIS/FUV-MAMA). When performing a "generic" search, you must include a wildcard or these more detailed results will be excluded.

We will demonstrate this by looking at the results for the query below:

In [ ]:
observations = Observations.query_criteria(proposal_pi="Espinoza, Nestor",
                                           instrument_name="NIRISS*")
observations

Our query returned many NIRISS observations led by the PI Dr. Espinoza. Let's get all the unique values under the `instrument_name` column to see what our `*` wildcard picked up.

In [ ]:
set(observations['instrument_name'])

Our observations have the advanced labeling; had we simply set `instrument_name = "NIRISS"`, we would have missed several observations. For more details on this advanced labeling, see the [JWST Instrument Names page](https://outerspace.stsci.edu/display/MASTDOCS/JWST+Instrument+Names).

#### A note of caution: There is such a thing as too many wildcards

You can be too generous with the wildcards, so be sure to exercise caution in their use. Too much ambiguity can lead to unintended results. Let's take a look at our example below.

In [ ]:
observations = Observations.query_criteria(proposal_pi='Espinoza, Nestor',
                                           instrument_name='NIR*') # Surely only one instrument begins with 'NIR'
set(observations['instrument_name'])

This query returns `NIRSPEC/SLIT` observations in addition to the NIRISS ones, which is not what we intended.

<a id="case2"></a>
### Case 2: Wildcard Search with `instrument_name` and `proposal_id`

Let's add an additional `string` criterion and wildcard into the mix. We'll do this with the `proposal_id` field which, despite its numeric content, is encoded as a string.

Let's query for a four digit proposal/program IDs that begin with `15`.

In [ ]:
observations = Observations.query_criteria(proposal_pi='Espinoza, Nestor',
                                           instrument_name='NIRISS*',
                                           proposal_id=['15%%']) # Only a four digit result will match this

set(observations['proposal_id']), set(observations['instrument_name'])

<a id="case3"></a>
### Case 3: Create a Moving Target Ephemeris using MAST Observations with Wildcard Search

We will be querying for image observations of Comet 67P Churyumov-Gerasimenko observed through the Hubble Space Telescope's Advanced Camera for Surveys (ACS) Wide Field Camera (WFC). This comet's name can be listed in different ways, so we will use `*` wildcards in our criteria query.

In [ ]:
observations = Observations.query_criteria(target_name="*67P*",
                                           instrument_name="ACS/WFC")

print(f"{len(observations)} total observations" + "\n")
print("Listed target names:")
print(set(observations['target_name']))

Above there are two names we get for Comet-67P. You should exercise caution when searching on the `target_name` criteria, since this is often entered by the PI who proposed the observation and can vary from person to person.

In the remainder of this notebook, we will construct a bare-bones ephemeris using the filtered MAST observations of this object and their metadata. We will then do some reverse engineering to query for the target based on coordinates using the ephemeris table, and hope that we get the same results back! Let's begin:

For simplicity, let's work only with the `'COMET-67P-CHURYUMOV-GERASIMENK'` observations to create our ephemeris.

In [ ]:
mask = observations["target_name"] == "COMET-67P-CHURYUMOV-GERASIMENK"
filtered_observations = observations[mask]
filtered_observations

Now that we have our filtered observations, let's sort the rows of this table based on the `t_min` criteria, which refers to the start time of the exposure in MJD.

In [ ]:
filtered_observations.sort("t_min")

Now that we've sorted our table, let's construct a basic ephemeris showing the path of our object over time (with `t_min`, or exposure start in MJD, as our time component):

In [ ]:
ephemeris = Table([filtered_observations["s_ra"],
                   filtered_observations["s_dec"],
                   filtered_observations["t_min"]], names=("ra", "dec", "t_min"))

Let's display the contents of our ephemeris, and use it to generate a plot of the comet's path:

In [ ]:
ephemeris.sort("t_min")
ephemeris

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(ephemeris[0:18]['ra'], ephemeris[0:18]['dec'], color='royalblue', s=200, lw=1., edgecolor='k')

plt.xlabel("Right Ascension (deg)", fontsize=20)
plt.ylabel("Declination (deg)", fontsize=20)

plt.title("Sky Coordinates", fontsize=20)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()

You can choose to save the ephemeris table in your current working directory by running the cell below:

In [ ]:
ephemeris.write('ephemeris-comet67p-cg.csv', format='ascii')  

## Resources

The following is a list of resources that were referenced throughout the tutorial, as well as some additional references that you may find useful:

* [`astroquery.mast.Observation` criteria queries](https://astroquery.readthedocs.io/en/latest/mast/mast.html#observation-criteria-queries)
* [JWST Instrument Names](https://outerspace.stsci.edu/display/MASTDOCS/JWST+Instrument+Names)

## Citations

If you use any of astroquery's tools for published research, please cite the authors. Follow this link for more information about citing astroquery:

* [Citing astroquery](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)

## About This Notebook

If you have comments or questions on this notebook, please contact us through the Archive Help Desk e-mail at archive@stsci.edu. <br>
<br>
Author: Jenny V. Medina <br>
Keywords: astroquery, wildcards, moving target <br>
Last Updated: Jun 2023